# Introduction to Antidromic Stimulation

## The Problem: Classifying Neurons by Their Projection Targets

Understanding brain function requires knowing not just *where* neurons are located, but *where they send their outputs*. In motor cortex, for example, some neurons project directly to the spinal cord to control movement (pyramidal tract neurons, PTNs), while others project to the basal ganglia to coordinate motor planning (corticostriatal neurons, CSNs). These cell types have distinct physiological properties and are differentially affected by neurological diseases like Parkinson's disease.

But here's the challenge: when you insert a recording electrode into motor cortex, you can only "see" the neuron's cell body and nearby processes. How do you determine where that neuron's axon goes potentially tens of millimeters away?

Several methods exist for neuron classification:
- **Anatomical tracing**: Inject tracers at target sites; neurons that project there become labeled
- **Optogenetics**: Express light-sensitive proteins in specific projection classes
- **Antidromic stimulation**: Stimulate at potential target sites and look for characteristic responses

This tutorial focuses on **antidromic stimulation**, the classical electrophysiological approach used extensively in primate neurophysiology. The data we analyze comes from a study by Pasquereau & Turner (2011, *Cerebral Cortex* 21:1362-1378) that used antidromic identification to compare the activity of PTNs and CSNs in parkinsonian monkeys. By separating these cell types, the researchers discovered that Parkinson's disease pathology selectively affects pyramidal tract neurons while relatively sparing corticostriatal neurons a finding that would be impossible without projection-based classification.

## Orthodromic vs Antidromic: The Direction of Action Potentials

Under normal physiological conditions, action potentials travel **orthodromically** - from the cell body (soma), down the axon, to the synaptic terminals at distant targets. This is the "forward" direction that carries information from the neuron to its downstream partners.

However, axons can conduct action potentials in either direction. If you artificially initiate an action potential at the axon terminal (by electrical stimulation), it will travel **antidromically** - backwards up the axon toward the cell body. When this antidromic spike reaches the soma, it can be detected by a recording electrode positioned there.

```
Orthodromic (normal physiology):     Cell Body ---> Axon ---> Target
Antidromic (experimental):           Cell Body <--- Axon <--- [STIM]
```

This bidirectionality is the key reason why antidromically detection works: if you stimulate at a particular brain region and a recorded neuron responds antidromically, you know that neuron's axon passes through (or terminates in) that region.

## Anatomical Context: M1, Cerebral Peduncle, and Striatum

In the Pasquereau & Turner study, recordings were made from **primary motor cortex (M1)** - the region of cortex most directly involved in generating voluntary movement commands. M1 contains multiple types of projection neurons, but two major classes were of interest:

### Pyramidal Tract Neurons (PTNs)
- **Location in cortex**: Layer 5b of M1 (deep layer, large pyramidal cells)
- **Projection target**: Spinal cord, via the corticospinal (pyramidal) tract
- **Axon pathway**: Axons descend through the corona radiata, enter the internal capsule, and converge in the **cerebral peduncle**: a massive white matter bundle at the base of the midbrain that carries all descending cortical motor commands
- **Function**: Direct control of spinal motor neurons and interneurons; execution of fine, skilled movements
- **Stimulation site for identification**: Cerebral peduncle (where PTN axons are densely bundled together)

### Corticostriatal Neurons (CSNs)
- **Location in cortex**: Layer 5 of M1 (intratelencephalic type)
- **Projection target**: Striatum (specifically the putamen, the motor division of the basal ganglia)
- **Axon pathway**: Axons project to the ipsilateral **posterolateral putamen**, which receives topographically organized inputs from motor cortices
- **Function**: Provide cortical information to basal ganglia circuits involved in action selection, motor learning, and movement vigor
- **Stimulation site for identification**: Posterolateral putamen (striatum)

By placing stimulating electrodes in both the cerebral peduncle and the putamen, researchers can determine which neurons are PTNs (respond to peduncle stimulation), which are CSNs (respond to putamen stimulation), and which are neither or both. The characteristic **latency** ( the time delay (in milliseconds) between the delivery of the electrical stimulus and the arrival of the resulting action potential at the recording electrode.) of the response also differs: PTNs have short latencies (0.6-2.5 ms) due to their large, fast-conducting axons, while CSNs have longer latencies (2-20 ms) due to their smaller, slower axons.

## Validating Antidromic Responses: The Three Criteria

Not every response to stimulation is antidromic. Electrical stimulation can activate neurons through multiple pathways: it might excite the target neuron's axon directly (antidromic), or it might activate local neurons at the stimulation site that then synaptically excite the recorded neuron (orthodromic/synaptic). Distinguishing these is critical.

The electrophysiology literature has established three standard criteria for confirming antidromic activation (Fuller & Schlag, 1976, *Brain Research* 112:283-298):

1. **Fixed latency**: The response occurs at a constant delay after stimulation. Axon conduction time is invariant (determined by axon length and conduction velocity), so antidromic latency shows minimal jitter (<0.5 ms). Synaptic responses, by contrast, have variable latency due to fluctuations in synaptic transmission.

2. **Frequency following**: The neuron can follow high-frequency stimulation (>200 Hz). Axons can fire rapidly, limited only by their refractory period (~1-2 ms). Synapses, however, fatigue at high frequencies due to vesicle depletion and receptor desensitization.

3. **Collision test**: A spontaneous spike can "collide with" and block the antidromic response. If a spontaneous action potential is traveling down the axon when the antidromic spike is traveling up, they meet and annihilate each other (both spikes enter refractory membrane). This only happens if both spikes are on the *same axon* - the definitive proof of antidromic activation.

In this tutorial, we will illustrate these three criteria using real antidromic testing data from the Pasquereau & Turner dataset, now publicly available on the DANDI Archive. By examining actual sweeps from identified PTNs and CSNs, you will see how these classical criteria work in practice.

## The Three Criteria for Antidromic Activation

Not every spike following stimulation is antidromic. Stimulation can also activate neurons *synaptically* (orthodromically) through connected circuits. To confirm antidromic activation, electrophysiologists verify three criteria:

| Criterion | Test | Why It Works |
|-----------|------|-------------|
| **Fixed latency** | Response occurs at constant delay across trials | Axon conduction time is invariant; synaptic delays fluctuate |
| **Frequency following** | Neuron follows stimulation at >200 Hz | Axons can fire rapidly; synapses fatigue at high frequencies |
| **Collision test** | Spontaneous spike blocks the antidromic response | Two action potentials annihilate when they meet on the same axon |

The **collision test** is the gold standard because it's the most definitive. Let's explore these criteria using real data from motor cortex recordings.

## Stream Data from DANDI

We'll use a session containing a corticostriatal neuron (CSN) identified through antidromic stimulation from the striatum. CSNs have longer antidromic latencies (2-20 ms) than PTNs (0.6-2.5 ms), making them easier to visualize.

In [ ]:
import h5py
import remfile
import numpy as np
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

# Connect to DANDI and get the dandiset
dandiset_id = "001636"
client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id, "draft")

# Select a session with a corticostriatal neuron that has BOTH protocol types
# This session has: 23 Collision sweeps + 8 FrequencyFollowing sweeps, latency 6.4 ms
asset_path = "sub-V/sub-V_ses-V++v4806++PostMPTP++Depth17700um++20000204_behavior+ecephys.nwb"
assets = list(dandiset.get_assets())
asset = next(a for a in assets if a.path == asset_path)
print(f"Streaming: {asset.path}")

# Stream the NWB file directly from DANDI (no download required)
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()
print(f"Session: {nwbfile.session_id}")

## Examine the Neuron Classification

The `units` table contains the classification results from antidromic testing. Let's see what we learned about this neuron.

In [ ]:
units_df = nwbfile.units.to_dataframe()

# Show key classification columns
classification_cols = [
    'neuron_projection_type',
    'antidromic_stimulation_sites',
    'antidromic_latency_ms',
    'antidromic_threshold'
]
units_df[classification_cols]

**Interpreting these parameters:**

| Parameter | Meaning |
|-----------|--------|
| `neuron_projection_type` | Classification based on which site evoked antidromic response |
| `antidromic_stimulation_sites` | Anatomical location where stimulation was delivered |
| `antidromic_latency_ms` | Time from stimulation to antidromic spike (conduction time) |
| `antidromic_threshold` | Minimum current (uA) needed to evoke response |

This neuron is classified as a **corticostriatal neuron** because it responded antidromically to stimulation in the posterolateral putamen (part of the striatum). The ~7 ms latency indicates a relatively slow conduction velocity, typical of the smaller-diameter axons of intratelencephalic-type cortical neurons.

## Access the Raw Antidromic Sweep Data

The classification above summarizes the result, but the NWB file also contains the raw waveforms from each stimulation trial. These are stored in the `AntidromicSweepsIntervals` table.

Each row represents one 50 ms "sweep" containing:
- The **neural response** (voltage recording from the cortical electrode)
- The **stimulation current** (current delivered to the striatum)

Sweeps are organized by protocol type: `Collision` or `FrequencyFollowing`.

In [ ]:
sweeps_df = nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()

print(f"Total sweeps: {len(sweeps_df)}")
print(f"\nSweeps by protocol type:")
print(sweeps_df['stimulation_protocol'].value_counts())
print(f"\nSweeps by stimulation location:")
print(sweeps_df['location'].value_counts())

In [ ]:
# Preview the sweep table structure
display_cols = ['unit_name', 'location', 'stimulation_protocol', 'sweep_number']
sweeps_df[display_cols].head(10)

### Helper Function for Extracting Sweep Data

Each sweep's `response` and `stimulation` columns contain references to specific sample ranges within consolidated time series. Here's a helper function to extract the waveforms:

In [ ]:
def get_sweep_waveforms(sweep_row):
    """
    Extract neural response and stimulation waveforms from a sweep.
    
    Returns:
        time_ms: Time axis in milliseconds (0 = stimulation onset)
        response_uv: Neural response in microvolts
        stim_ua: Stimulation current in microamperes
    """
    # Extract response data
    response_ref = sweep_row['response']
    index_start, count, response_series = response_ref
    response_uv = response_series.data[index_start:index_start + count].flatten() * response_series.conversion * 1e6
    
    # Extract stimulation data
    stim_ref = sweep_row['stimulation']
    index_start_s, count_s, stim_series = stim_ref
    stim_ua = stim_series.data[index_start_s:index_start_s + count_s].flatten() * stim_series.conversion * 1e6
    
    # Create time axis: sweeps are 50ms, centered on stimulation (t=0 at 25ms)
    sampling_rate = response_series.rate
    time_ms = (np.arange(count) / sampling_rate - 0.025) * 1000
    
    return time_ms, response_uv, stim_ua

## Visualize a Single Sweep

Let's examine what happens when we stimulate the striatum and record from the motor cortex. The key features to identify:

1. **Stimulation artifact** (t = 0): A large, brief deflection from the electrical stimulus
2. **Antidromic response** (t = latency): A smaller waveform when the action potential arrives at the cell body

In [ ]:
# Get the expected latency from the units table
antidromic_latency = units_df['antidromic_latency_ms'].iloc[0]

# Plot a single sweep
sweep = sweeps_df.iloc[0]
time_ms, response_uv, stim_ua = get_sweep_waveforms(sweep)

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Top panel: Neural response
axes[0].plot(time_ms, response_uv, 'b-', linewidth=0.8)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
axes[0].axvline(antidromic_latency, color='green', linestyle=':', linewidth=2, label=f'Expected response ({antidromic_latency} ms)')
axes[0].set_ylabel('Neural Response (uV)', fontsize=12)
axes[0].set_title(f'Single Antidromic Sweep - {sweep["stimulation_protocol"]} Protocol', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=10)
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)

# Bottom panel: Stimulation current
axes[1].plot(time_ms, stim_ua, 'r-', linewidth=0.8)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Time relative to stimulation (ms)', fontsize=12)
axes[1].set_ylabel('Stim Current (uA)', fontsize=12)
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

print(f"\nKey observations:")
print(f"  - Large deflection at t=0 is the stimulation artifact (electrical contamination)")
print(f"  - Smaller waveform at ~{antidromic_latency} ms is the antidromic spike")
print(f"  - Biphasic stimulation pulse minimizes tissue damage")

### What We See in This Sweep

**Top panel (Neural Response):**
- At **t = 0 ms** (red dashed line), there is a large, sharp deflection. This is the **stimulation artifact** - electrical contamination from the stimulation pulse that travels through the tissue and is picked up by the recording electrode. This is NOT neural activity; it's purely electrical.
- At **t = 6.4 ms** (green dotted line), there is a smaller but distinct waveform. This is the **antidromic spike** - the action potential that was initiated at the striatum, traveled backwards up the axon, and arrived at the cell body in motor cortex where we're recording.
- The time between stimulation and response (6.4 ms) represents the **conduction time** - how long it takes the action potential to travel from the stimulation site to the recording site.

**Bottom panel (Stimulation Current):**
- The biphasic waveform shows the current delivered to the striatum: first a positive phase, then a negative phase of equal magnitude.
- This charge-balanced pulse minimizes tissue damage during repeated stimulation.

**Key insight:** The clear separation between the artifact (t=0) and the response (t=6.4 ms) allows us to distinguish the electrical contamination from the true neural signal. This separation is larger for CSNs (2-20 ms) than PTNs (0.6-2.5 ms), which is why CSNs are easier to study.

### Why Biphasic Stimulation?

Notice in the bottom panel that the stimulation current has two phases: first positive, then negative (or vice versa). This is called **biphasic stimulation**.

```
Current
   ^
   |    +250 uA
   |    ______
   |   |      |
---+---|      |---------- Time
   |   |      |______|
   |          -250 uA
```

**Why not just use a single pulse (monophasic)?**

| Issue | Monophasic | Biphasic |
|-------|------------|----------|
| **Charge accumulation** | Net charge builds up at electrode | Positive and negative phases cancel out |
| **Tissue damage** | pH changes, toxic byproducts | Minimal - charge balanced |
| **Electrode corrosion** | Electrochemical degradation | Reduced - no net current flow |

Biphasic pulses are standard for any repeated neural stimulation (deep brain stimulation, cochlear implants, research applications). The equal-and-opposite phases ensure no net charge transfer, protecting both the tissue and the electrode.

## Criterion 1: Fixed Latency (Frequency Following Test)

The frequency following test delivers repeated stimulation pulses and checks whether the neuron responds at the same latency each time. Antidromic responses have **invariant latency** because axon conduction time is constant.

In contrast, synaptic responses have variable latency due to:
- Synaptic transmission delays that fluctuate
- Variable numbers of intervening synapses
- Neurotransmitter release stochasticity

Let's overlay multiple frequency following sweeps to see the latency consistency.

In [ ]:
# Get frequency following sweeps
ff_sweeps = sweeps_df[sweeps_df['stimulation_protocol'] == 'FrequencyFollowing']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: Overlay all sweeps
ax = axes[0]
for i in range(len(ff_sweeps)):
    sweep = ff_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.5)

ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2, 
           label=f'Expected latency ({antidromic_latency} ms)')
ax.set_xlim(-2, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=12)
ax.set_ylabel('Neural Response (uV)', fontsize=12)
ax.set_title(f'All Sweeps Overlaid (n={len(ff_sweeps)})', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right panel: Waterfall view showing individual sweeps
ax = axes[1]
offset = 0
spacing = 60  # uV between traces

for i in range(len(ff_sweeps)):
    sweep = ff_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv + offset, 'b-', linewidth=0.8)
    ax.text(-3, offset, f'#{i+1}', fontsize=8, va='center')
    offset += spacing

ax.axvline(0, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=1.5, alpha=0.7)
ax.set_xlim(-5, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=12)
ax.set_title('Individual Sweeps (Waterfall View)', fontsize=13, fontweight='bold')
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()

print(f"Key observation: Every single sweep shows a response at exactly {antidromic_latency} ms.")
print(f"This FIXED LATENCY is the hallmark of antidromic activation.")
print(f"\nWhy? Because: Latency = Distance / Conduction Velocity = CONSTANT")

### What We See: Evidence for Fixed Latency

**Left panel (Overlay):**
- All 8 frequency following sweeps are plotted on top of each other
- Notice how the responses form a **tight cluster** at exactly 6.4 ms after stimulation
- The stimulation artifacts at t=0 also overlap precisely (as expected - same stimulus each time)
- If latency were variable, we would see the responses spread out horizontally, forming a "smear" rather than a tight cluster

**Right panel (Waterfall):**
- Each sweep is plotted separately, offset vertically for clarity
- You can trace a vertical line at 6.4 ms and see that **every single sweep** has its response aligned to that time
- This "stacking" visualization makes it easy to see that the response timing is identical across trials

**Why this matters:**
- Axon conduction velocity is a physical property determined by axon diameter and myelination
- The distance from striatum to motor cortex is fixed anatomically
- Therefore: **Latency = Distance / Velocity = CONSTANT**

If this were synaptic activation instead:
- Synaptic vesicle release is probabilistic (variable delay)
- Multiple synapses in series add cumulative jitter
- Postsynaptic integration time varies
- We would see latencies spread over 1-5 ms or more

### Quantifying Latency Consistency

The overlay and waterfall views show qualitatively that responses occur at the same time. But how consistent is the latency *quantitatively*? Let's measure the actual response latency in each sweep and plot the distribution.

In [ ]:
from scipy.signal import find_peaks

def measure_response_latency(time_ms, response_uv, expected_latency, window_ms=2.0):
    """Measure actual response latency by finding the peak near expected latency."""
    mask = (time_ms >= expected_latency - window_ms) & (time_ms <= expected_latency + window_ms)
    if not mask.any():
        return np.nan
    
    window_response = np.abs(response_uv[mask])
    window_time = time_ms[mask]
    
    peaks, properties = find_peaks(window_response, height=np.percentile(window_response, 70))
    if len(peaks) == 0:
        return np.nan
    
    highest_peak_idx = peaks[np.argmax(properties['peak_heights'])]
    return window_time[highest_peak_idx]

# Measure latency from ALL sweeps (both protocols demonstrate fixed latency)
all_sweeps = sweeps_df.copy()
measured_latencies = []

for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    lat = measure_response_latency(time_ms, response_uv, antidromic_latency)
    if not np.isnan(lat):
        measured_latencies.append(lat)

# Plot latency histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Left: Zoomed view of response window
ax = axes[0]
for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.5, alpha=0.4)

ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2)
ax.axvspan(antidromic_latency - 0.5, antidromic_latency + 0.5, alpha=0.2, color='green',
           label='Expected +/- 0.5 ms')
ax.set_xlim(antidromic_latency - 3, antidromic_latency + 3)
ax.set_xlabel('Time (ms)', fontsize=11)
ax.set_ylabel('Response (uV)', fontsize=11)
ax.set_title('Zoomed View of Response Window', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right: Latency histogram
ax = axes[1]
ax.hist(measured_latencies, bins=15, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2,
           label=f'Expected: {antidromic_latency} ms')
ax.axvline(np.mean(measured_latencies), color='red', linestyle='--', linewidth=2,
           label=f'Measured mean: {np.mean(measured_latencies):.2f} ms')
ax.set_xlabel('Measured Latency (ms)', fontsize=11)
ax.set_ylabel('Count', fontsize=11)
latency_std = np.std(measured_latencies)
ax.set_title(f'Latency Distribution (SD = {latency_std:.3f} ms)', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()

print(f"Latency statistics from {len(measured_latencies)} sweeps:")
print(f"  Mean:     {np.mean(measured_latencies):.3f} ms")
print(f"  SD:       {np.std(measured_latencies):.3f} ms")
print(f"  Range:    {np.min(measured_latencies):.3f} - {np.max(measured_latencies):.3f} ms")
print(f"  Expected: {antidromic_latency} ms")
print(f"\nThe extremely low SD ({latency_std:.3f} ms) confirms FIXED LATENCY.")
print("Synaptic responses typically show SD > 1 ms due to variable transmission delays.")

### Interpreting the Latency Statistics

**Left panel (Zoomed Response Window):**
- This zooms in on a +/- 3 ms window around the expected latency (6.4 ms)
- The green shaded region shows +/- 0.5 ms around the expected value
- Notice that virtually all response peaks fall within or very close to this narrow band
- The waveform shapes are also highly consistent - same neuron, same response each time

**Right panel (Latency Histogram):**
- We measured the actual peak latency in each of the 31 sweeps
- The histogram shows how these measurements are distributed
- **Mean = 6.68 ms**: Very close to the expected 6.4 ms (small systematic offset from peak detection)
- **SD = 0.637 ms**: Extremely low variability!
- The tight, unimodal distribution confirms fixed latency

**Putting the numbers in context:**

| Activation Type | Typical Latency SD |
|-----------------|-------------------|
| **Antidromic** | < 0.5 ms |
| **Monosynaptic** | 0.5 - 1.5 ms |
| **Polysynaptic** | 1.5 - 5+ ms |

Our measured SD of 0.637 ms is consistent with antidromic activation. The small amount of variability we do see likely comes from:
- Measurement noise in detecting the exact peak
- Minor fluctuations in axon excitability
- Recording system timing jitter

This is **not** the multi-millisecond jitter we would expect from synaptic transmission.

## Criterion 2: The Collision Test (Gold Standard)

The collision test is the most definitive verification of antidromic activation. It exploits a fundamental property of axons: **two action potentials traveling in opposite directions on the same axon will collide and annihilate each other**.

### Switching to a Better Example Session

To best illustrate the collision test, we'll use a different session that shows more dramatic amplitude variability between blocked and unblocked responses. This is a key advantage of the NWB/DANDI ecosystem - we can easily access multiple sessions to find the clearest examples for teaching.

### How the Collision Test Works

```
Normal antidromic response (no collision):
    
    Cell Body <----[antidromic spike]---- [STIM at striatum]
         |                                      |
         V                                      |
    Recorded spike                              |


Collision (spontaneous spike blocks response):

    Cell Body ---[spontaneous spike]---> 
                                    COLLISION!
              <---[antidromic spike]--- [STIM]
         |                                      
         V                                      
    NO recorded spike (blocked)                 
```

### The Critical Collision Window

For collision to occur, the spontaneous spike must be traveling down the axon when the antidromic spike is traveling up. This defines a **critical window** before stimulation:

```
Critical window = Antidromic latency + Axonal refractory period
```

If a spontaneous spike occurs within this window before stimulation, the antidromic response should be blocked.

In [ ]:
# Load a different session with clearer collision effects
# Session v4518 has amplitude CV=60% - much better for demonstrating collision blocking
collision_asset_path = "sub-V/sub-V_ses-V++v4518++PostMPTP++Depth19900um++20000124_behavior+ecephys.nwb"
collision_asset = next(a for a in assets if a.path == collision_asset_path)
print(f"Loading collision example: {collision_asset.path.split('/')[-1]}")

collision_s3_url = collision_asset.get_content_url(follow_redirects=1, strip_query=False)
collision_file = h5py.File(remfile.File(collision_s3_url), mode="r")
collision_io = NWBHDF5IO(file=collision_file)
collision_nwbfile = collision_io.read()

# Get data from this session
collision_units_df = collision_nwbfile.units.to_dataframe()
collision_sweeps_df = collision_nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()

collision_latency = collision_units_df['antidromic_latency_ms'].iloc[0]
print(f"Neuron type: {collision_units_df['neuron_projection_type'].iloc[0]}")
print(f"Antidromic latency: {collision_latency} ms")

# Get collision sweeps
collision_sweeps = collision_sweeps_df[collision_sweeps_df['stimulation_protocol'] == 'Collision']
print(f"Collision sweeps: {len(collision_sweeps)}")

# Calculate critical window
refractory_period = 2.0  # ms
critical_window = collision_latency + refractory_period
print(f"Critical collision window: {critical_window:.1f} ms")

# Measure response amplitude for each sweep
def measure_amplitude(sweep_row, latency, window_ms=1.5):
    time_ms, response_uv, _ = get_sweep_waveforms(sweep_row)
    mask = (time_ms >= latency - window_ms) & (time_ms <= latency + window_ms)
    if mask.any():
        return np.max(response_uv[mask]) - np.min(response_uv[mask])
    return 0

amplitudes = [measure_amplitude(collision_sweeps.iloc[i], collision_latency) 
              for i in range(len(collision_sweeps))]
median_amp = np.median(amplitudes)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: All sweeps with critical window
ax = axes[0]
ax.axvspan(-critical_window, 0, alpha=0.2, color='orange', label=f'Critical window ({critical_window:.1f} ms)')

for i in range(len(collision_sweeps)):
    sweep = collision_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.5, alpha=0.3)

ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2, 
           label=f'Expected response ({collision_latency} ms)')
ax.set_xlim(-15, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=12)
ax.set_ylabel('Neural Response (uV)', fontsize=12)
ax.set_title(f'All Collision Sweeps (n={len(collision_sweeps)})', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right panel: Response amplitude distribution
ax = axes[1]
ax.hist(amplitudes, bins=12, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(median_amp, color='red', linestyle='--', linewidth=2, 
           label=f'Median: {median_amp:.1f} uV')
ax.set_xlabel('Response Amplitude (uV)', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Response Amplitude Distribution', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()

amp_cv = np.std(amplitudes) / np.mean(amplitudes) * 100
print(f"\nResponse amplitude statistics:")
print(f"  Range: {np.min(amplitudes):.1f} - {np.max(amplitudes):.1f} uV")
print(f"  CV: {amp_cv:.1f}%")
print(f"  This high variability reflects the collision test in action!")

### Interpreting the Collision Test Results

**Left panel (All Sweeps):**
- The **orange shaded region** shows the critical collision window (~8.8 ms before stimulation)
- Looking at the responses at 6.75 ms, notice that some sweeps have **strong deflections** while others appear **weaker or absent**
- This variability is exactly what we expect from a collision test!
- In the pre-stimulus period (orange region), you may notice occasional deflections - these could be spontaneous spikes that caused collisions

**Right panel (Amplitude Histogram):**
- The histogram shows a **wide spread** of response amplitudes (range: ~13-64 uV)
- This is very different from the fixed latency test, where we wanted consistency
- The **high coefficient of variation (CV ~60%)** indicates substantial variability
- The distribution is spread out rather than tightly clustered

**Why this variability is the key finding:**

| Amplitude | Interpretation |
|-----------|---------------|
| **High (>30 uV)** | No collision occurred - antidromic spike reached cell body |
| **Low (<20 uV)** | Likely collision - spontaneous spike blocked the antidromic response |
| **Middle** | Partial collision or measurement variability |

This variability **proves** antidromic activation because:
- Synaptic responses would NOT be blocked by spontaneous activity
- Only a spike traveling on the **same axon** can collide with the antidromic spike
- The physics is unavoidable: two action potentials meeting head-on annihilate each other

### Comparing Blocked vs Unblocked Responses

If collision is occurring, we should see some sweeps with strong responses (no collision - the antidromic spike reached the cell body) and others with weak or absent responses (collision occurred - the spike was blocked). Let's separate sweeps by amplitude to see this effect:

In [ ]:
# Separate sweeps by amplitude (above vs below median)
high_amp_indices = [i for i, amp in enumerate(amplitudes) if amp >= median_amp]
low_amp_indices = [i for i, amp in enumerate(amplitudes) if amp < median_amp]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: High amplitude sweeps (clear responses - no collision)
ax = axes[0]
ax.axvspan(-critical_window, 0, alpha=0.15, color='orange')

for idx in high_amp_indices:
    sweep = collision_sweeps.iloc[idx]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.5)

ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2)
ax.set_xlim(-15, 15)
ax.set_xlabel('Time (ms)', fontsize=11)
ax.set_ylabel('Response (uV)', fontsize=11)
ax.set_title(f'Clear Responses (n={len(high_amp_indices)}) - No Collision', fontsize=13, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add annotation
ax.text(0.02, 0.98, 'No spontaneous spike\nin critical window\n-> Response present',
        transform=ax.transAxes, fontsize=9, va='top',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

# Right: Low amplitude sweeps (potential collisions)
ax = axes[1]
ax.axvspan(-critical_window, 0, alpha=0.15, color='orange')

for idx in low_amp_indices:
    sweep = collision_sweeps.iloc[idx]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.5)

ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2)
ax.set_xlim(-15, 15)
ax.set_xlabel('Time (ms)', fontsize=11)
ax.set_ylabel('Response (uV)', fontsize=11)
ax.set_title(f'Reduced Responses (n={len(low_amp_indices)}) - Potential Collisions', fontsize=13, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add annotation
ax.text(0.02, 0.98, 'Spontaneous spike in\ncritical window?\n-> Response blocked',
        transform=ax.transAxes, fontsize=9, va='top',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()

print("Direct comparison of blocked vs unblocked responses:")
print(f"  High amplitude sweeps: {len(high_amp_indices)} (mean: {np.mean([amplitudes[i] for i in high_amp_indices]):.1f} uV)")
print(f"  Low amplitude sweeps:  {len(low_amp_indices)} (mean: {np.mean([amplitudes[i] for i in low_amp_indices]):.1f} uV)")

### Side-by-Side: The Collision Effect Visualized

**Left panel (Clear Responses - No Collision):**
- These sweeps have **high amplitude** responses at the expected latency
- The waveforms are prominent and consistent
- Looking at the pre-stimulus period (orange), there is relatively **little activity**
- Interpretation: No spontaneous spike occurred in the critical window, so the antidromic spike traveled unimpeded to the cell body

**Right panel (Reduced Responses - Potential Collisions):**
- These sweeps have **low amplitude** or nearly absent responses
- The waveforms at the expected latency time are much smaller or missing
- Some sweeps may show activity in the pre-stimulus period
- Interpretation: A spontaneous spike likely occurred in the critical window and collided with the antidromic spike

**The contrast between these panels is the collision test in action:**

This is the most definitive test because it exploits an unavoidable physical phenomenon. If the neuron were being activated synaptically (through chemical transmission), spontaneous spikes would NOT block the response - they travel on different paths. Only when the orthodromic and antidromic spikes are on the **same axon** can collision occur.

**Note on interpretation:** In a formal collision test protocol, the experimenter deliberately times the stimulation to occur shortly after detecting a spontaneous spike. Here, we're seeing naturally-occurring collisions based on the neuron's spontaneous firing rate. The principle is the same.

## Summary of Sessions Used

In this tutorial, we demonstrated the antidromic criteria using two different sessions from the same dataset, each chosen because it provided the clearest example:

| Criterion | Session | Why This Session |
|-----------|---------|------------------|
| **Fixed Latency** | v4806 | 31 sweeps with consistent responses, latency SD = 0.64 ms |
| **Collision Test** | v4518 | 25 collision sweeps with high amplitude variability (CV = 60%) |

This illustrates a key advantage of standardized data formats like NWB combined with repositories like DANDI: you can easily access multiple datasets to find the best examples for your analysis or teaching purposes.

## PTN vs CSN: Two Projection Types

Motor cortex contains neurons with different axonal targets. In this dataset, two main projection types were identified:

| Property | Pyramidal Tract Neurons (PTNs) | Corticostriatal Neurons (CSNs) |
|----------|-------------------------------|--------------------------------|
| **Projection target** | Spinal cord via cerebral peduncle | Striatum (putamen) |
| **Stimulation site** | Cerebral peduncle | Posterolateral putamen |
| **Antidromic latency** | 0.6-2.5 ms | 2-20 ms |
| **Axon diameter** | Large (fast conduction) | Smaller (slower conduction) |
| **Conduction velocity** | 20-50 m/s | 2-10 m/s |
| **Cortical layer** | Layer 5b (large pyramidal) | Layer 5 (intratelencephalic) |
| **Function** | Direct motor commands to spinal cord | Cortical input to basal ganglia |

**Why the latency difference?**

PTNs have large-diameter, heavily myelinated axons optimized for rapid, reliable motor commands. CSNs have smaller axons typical of intratelencephalic-type cortical neurons that coordinate with basal ganglia circuits.

In [ ]:
# Display the classification for this neuron
unit = units_df.iloc[0]

print("This neuron's classification:")
print(f"  Type: {unit['neuron_projection_type']}")
print(f"  Stimulation site: {unit['antidromic_stimulation_sites']}")
print(f"  Latency: {unit['antidromic_latency_ms']} ms")
print(f"  Threshold: {unit['antidromic_threshold']} uA")

# Estimate conduction velocity
# Approximate distance from M1 to striatum: ~30 mm
distance_mm = 30
conduction_velocity = distance_mm / unit['antidromic_latency_ms']  # m/s
print(f"  Estimated conduction velocity: ~{conduction_velocity:.1f} m/s")

## Summary

**What we learned:**

1. **Antidromic stimulation** allows classification of neurons by their axonal projection targets by stimulating at the target and recording the backwards-traveling action potential

2. **Three criteria** distinguish antidromic from synaptic activation:
   - Fixed latency (axon conduction time is invariant)
   - Frequency following (axons can fire rapidly; synapses fatigue)
   - Collision (spontaneous and antidromic spikes annihilate)

3. **Cell type differences** reflect axonal properties:
   - PTNs: Short latency (0.6-2.5 ms), large fast axons to spinal cord
   - CSNs: Longer latency (2-20 ms), smaller slower axons to striatum

4. **In NWB format**, antidromic data is stored in:
   - `units` table: Classification results (type, latency, threshold)
   - `AntidromicSweepsIntervals`: Raw waveforms for each test sweep

**Further reading:**
- Pasquereau B, Turner RS (2011) Primary motor cortex of the parkinsonian monkey. *Cerebral Cortex* 21:1362-1378.
- Fuller JH, Schlag JD (1976) Determination of antidromic excitation by the collision test. *Brain Research* 112:283-298.